In [2]:
pip -q install autogluon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 13.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.2/266.2 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.1/64.1 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 430.0/430.0 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 352.2/352.2 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.7/174.7 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.2/68.2 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.7/196.7 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.0/71.0 kB 4.8 MB/s eta 0:0

In [3]:
from autogluon.tabular import TabularDataset, TabularPredictor

In [4]:
import pandas as pd

# start

In [5]:
train_data = pd.read_csv("/kaggle/input/hearth-disease-recognition/train.csv")

In [6]:
train_data['History of HeartDisease or Attack'].value_counts()

History of HeartDisease or Attack
No     203322
Yes     18068
Name: count, dtype: int64

# preprocess

In [7]:
# delete target null
print(len(train_data))
train_data = train_data.dropna(subset=['History of HeartDisease or Attack'])
print(len(train_data))

223084
221390


In [8]:
train_data["Age"].describe()

count    221390.000000
mean         54.660215
std          17.773171
min          18.000000
25%          42.000000
50%          56.000000
75%          67.000000
max         100.000000
Name: Age, dtype: float64

In [9]:
import numpy as np
import pandas as pd

def create_new_features(train_data):
    # """
    # This function creates 10 new features from the existing columns in train_data.
    # It assumes that many binary columns are coded as "Yes"/"No" and converts them into 1/0.
    # It also maps categorical education and income levels into numeric scores.
    # """
    # --- Helper: Convert Yes/No to binary ---
    binary_map = {"Yes": 1, "No": 0}
    binary_cols = [
        "High Blood Pressure", "Told High Cholesterol", "Cholesterol Checked",
        "Smoked 100+ Cigarettes", "Diagnosed Stroke", "Diagnosed Diabetes",
        "Leisure Physical Activity", "Heavy Alcohol Consumption",
        "Health Care Coverage", "Doctor Visit Cost Barrier",
        "Difficulty Walking", "Vegetable or Fruit Intake (1+ per Day)"
    ]
    
    for col in binary_cols:
        # Create a new binary column with suffix '_bin'
        if train_data[col].dtype == object:
            train_data[col + "_bin"] = train_data[col].map(binary_map)
        else:
            train_data[col + "_bin"] = train_data[col]


    # --------Feature 2 ------------
    def categorize_bmi(bmi): # GOOD 
        if bmi < 18.5:
            return "Underweight"
        elif bmi < 25:
            return "Normal weight"
        elif bmi < 30:
            return "Overweight"
        else:
            return "Obese"

    train_data['BMI Category'] = train_data['Body Mass Index'].apply(categorize_bmi)

    # # Derived Feature: Obesity Risk # may be good, GOOD
    train_data['Obesity Risk'] = ((train_data['Body Mass Index'] >= 30) & (train_data['Leisure Physical Activity'] == 'No')).astype(int)

    # #Drop column
    columns_to_drop = [
        "High Blood Pressure_bin", "Told High Cholesterol_bin", "Cholesterol Checked_bin",
        "Smoked 100+ Cigarettes_bin", "Diagnosed Stroke_bin", "Diagnosed Diabetes_bin",
        "Leisure Physical Activity_bin", "Heavy Alcohol Consumption_bin",
        "Health Care Coverage_bin", "Doctor Visit Cost Barrier_bin",
        "Difficulty Walking_bin", "Vegetable or Fruit Intake (1+ per Day)_bin",
        "Leisure Physical Activity",
    ]
    train_data = train_data.drop(columns=columns_to_drop)

    


    return train_data



In [10]:
train_data = create_new_features(train_data)
train_data.head(3)

/usr/local/lib/python3.10/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in greater_equal
  return op(a, b)


,ID,History of HeartDisease or Attack,High Blood Pressure,Told High Cholesterol,Cholesterol Checked,Body Mass Index,Smoked 100+ Cigarettes,Diagnosed Stroke,Diagnosed Diabetes,Heavy Alcohol Consumption,...,Doctor Visit Cost Barrier,General Health,Difficulty Walking,Sex,Education Level,Income Level,Age,Vegetable or Fruit Intake (1+ per Day),BMI Category,Obesity Risk
0,train_000001,No,Yes,Yes,Yes,40.68,Yes,No,No,No,...,No,Very Poor,Yes,Female,High school graduate,"$15,000 to less than $20,000",64,Yes,Obese,1
1,train_000002,No,No,No,No,24.36,Yes,No,No,No,...,Yes,Fair,No,Female,College graduate,"Less than $10,000",50,No,Normal weight,0
2,train_000003,No,Yes,Yes,Yes,27.33,No,No,No,No,...,Yes,Very Poor,Yes,Female,High school graduate,"$75,000 or more",61,Yes,Overweight,0


In [11]:
train_data.iloc[0]

ID                                                        train_000001
History of HeartDisease or Attack                                   No
High Blood Pressure                                                Yes
Told High Cholesterol                                              Yes
Cholesterol Checked                                                Yes
Body Mass Index                                                  40.68
Smoked 100+ Cigarettes                                             Yes
Diagnosed Stroke                                                    No
Diagnosed Diabetes                                                  No
Heavy Alcohol Consumption                                           No
Health Care Coverage                                               Yes
Doctor Visit Cost Barrier                                           No
General Health                                               Very Poor
Difficulty Walking                                                 Yes
Sex   

In [12]:
df_yes = train_data[train_data['History of HeartDisease or Attack'] == 'Yes'].sample(n=18068, random_state=42)
df_no = train_data[train_data['History of HeartDisease or Attack'] == 'No'].sample(n=20000, random_state=42)
train_data = pd.concat([df_yes, df_no], ignore_index=True)

In [13]:
train_data.head(3)

,ID,History of HeartDisease or Attack,High Blood Pressure,Told High Cholesterol,Cholesterol Checked,Body Mass Index,Smoked 100+ Cigarettes,Diagnosed Stroke,Diagnosed Diabetes,Heavy Alcohol Consumption,...,Doctor Visit Cost Barrier,General Health,Difficulty Walking,Sex,Education Level,Income Level,Age,Vegetable or Fruit Intake (1+ per Day),BMI Category,Obesity Risk
0,train_183649,Yes,Yes,Yes,Yes,26.36,No,Yes,No,No,...,No,Very Poor,Yes,Female,College graduate,"$75,000 or more",47,Yes,Overweight,0
1,train_001613,Yes,Yes,Yes,Yes,26.11,Yes,Yes,Yes,No,...,No,Poor,No,Female,Some high school,"($10,000 to less than $15,000",59,Yes,Overweight,0
2,train_072658,Yes,No,Yes,Yes,26.23,No,No,No,No,...,Yes,Poor,Yes,Female,College graduate,"$75,000 or more",58,Yes,Overweight,0


In [14]:
train_data['History of HeartDisease or Attack'].value_counts()
# train_data['General Health'].isna().sum()

History of HeartDisease or Attack
No     20000
Yes    18068
Name: count, dtype: int64

# model time

In [23]:
save_path = 'best_model'
hyperparameters = {
    'GBM': [
        {'ag_args_fit': {'num_gpus': 0}},  # Train with CPU
        {'ag_args_fit': {'num_gpus': 1}}   # Train with GPU
    ],
    'CAT': [
        {'ag_args_fit': {'num_gpus': 0}},  # Train with CPU
        {'ag_args_fit': {'num_gpus': 1}}   # Train with GPU
    ],
    'XGB': [
        {'ag_args_fit': {'num_gpus': 0}},  # Train with CPU
        {'ag_args_fit': {'num_gpus': 1}}   # Train with GPU
    ],
    'RF': [
        {'ag_args_fit': {'num_gpus': 0}},  # Train with CPU
        {'ag_args_fit': {'num_gpus': 1}}   # Train with GPU
    ],

}
time_limit=600

In [24]:
predictor = TabularPredictor(label='History of HeartDisease or Attack',
                            problem_type='binary',
                            path=save_path,
                            # eval_metric='f1_macro'
                            ).fit(
                                train_data,
                                presets='best_quality',
                                hyperparameters=hyperparameters,
                                time_limit=time_limit
                            )
print("Finish!")

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sun Nov 10 10:07:59 UTC 2024
CPU Count:          4
Memory Avail:       29.34 GB / 31.35 GB (93.6%)
Disk Space Avail:   19.50 GB / 19.52 GB (99.9%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets of the data. Then holdout validation data is used to detect stacked 

Finish!


# test data

In [26]:
test_data = pd.read_csv("/kaggle/input/hearth-disease-recognition/test.csv")
test_data.head()

,ID,High Blood Pressure,Told High Cholesterol,Cholesterol Checked,Body Mass Index,Smoked 100+ Cigarettes,Diagnosed Stroke,Diagnosed Diabetes,Leisure Physical Activity,Heavy Alcohol Consumption,Health Care Coverage,Doctor Visit Cost Barrier,General Health,Difficulty Walking,Sex,Education Level,Income Level,Age,Vegetable or Fruit Intake (1+ per Day)
0,test_000001,Yes,Yes,Yes,24.84,No,No,No,Yes,No,Yes,No,Good,No,Female,Some college or technical school,"$20,000 to less than $25,000",71,Yes
1,test_000002,Yes,No,Yes,29.08,Yes,No,No,No,No,Yes,No,Fair,No,Female,College graduate,"$50,000 to less than $75,000",61,No
2,test_000003,Yes,Yes,Yes,35.23,Yes,No,No,No,No,Yes,No,Fair,Yes,Female,Some college or technical school,"Less than $10,000",67,Yes
3,test_000004,No,No,Yes,24.78,Yes,No,No,No,No,Yes,No,Fair,No,Female,Some college or technical school,"$50,000 to less than $75,000",50,Yes
4,test_000005,No,No,Yes,27.57,Yes,No,No,No,No,Yes,No,Fair,No,Male,Some college or technical school,"$25,000 to less than $35,000",40,Yes


In [27]:
for i in test_data.columns:
    print(i, end=" ")
    print(test_data[i].isna().sum())

ID 0
High Blood Pressure 0
Told High Cholesterol 0
Cholesterol Checked 0
Body Mass Index 0
Smoked 100+ Cigarettes 0
Diagnosed Stroke 0
Diagnosed Diabetes 0
Leisure Physical Activity 0
Heavy Alcohol Consumption 0
Health Care Coverage 0
Doctor Visit Cost Barrier 0
General Health 0
Difficulty Walking 0
Sex 0
Education Level 0
Income Level 0
Age 0
Vegetable or Fruit Intake (1+ per Day) 0


In [28]:
# preprocess test
test_data = create_new_features(test_data)

In [29]:
# y_pred = predictor.predict(test_data)
# y_pred.head() predict_proba

# use proba
y_pred = predictor.predict_proba(test_data)
y_pred.head()

,No,Yes
0,0.540053,0.459947
1,0.676904,0.323096
2,0.302197,0.697803
3,0.863165,0.136835
4,0.848005,0.151995


In [30]:
y_pred['outcome'] = y_pred.apply(lambda row: 'Yes' if row['Yes'] > 0.505 else 'No', axis=1)
y_pred['outcome'].value_counts()

outcome
No     50155
Yes    24206
Name: count, dtype: int64

In [31]:
sample_submission = pd.read_csv("/kaggle/input/hearth-disease-recognition/sample_submission.csv")
sample_submission.head(2)

,ID,History of HeartDisease or Attack
0,test_000001,No
1,test_000002,No


In [32]:
# sample_submission['History of HeartDisease or Attack'] = y_pred
sample_submission['History of HeartDisease or Attack'] = y_pred['outcome']
sample_submission.tail()

,ID,History of HeartDisease or Attack
74356,test_074357,No
74357,test_074358,Yes
74358,test_074359,Yes
74359,test_074360,Yes
74360,test_074361,No


In [33]:
# sample_submission.to_csv("preprocess_first__no_prepare_52_7%confident_Accuracy_train20k_20k.csv", index=False)
sample_submission.to_csv("submission.csv", index=False)

# check submission

In [34]:
wowow = pd.read_csv("/kaggle/working/submission.csv")
wowow['History of HeartDisease or Attack'].value_counts()

History of HeartDisease or Attack
No     50155
Yes    24206
Name: count, dtype: int64